In [1]:
import pandas as pd
import numpy as np 
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
from datetime import datetime
import os
import random 
import seaborn as sns
import functools
from fbprophet import Prophet
from statistics import mean
from sklearn.metrics import r2_score

ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [2]:
data = pd.read_csv(os.path.expanduser("~/Desktop/DS_Competition/ucsbdata.csv")).dropna(axis=0)
data['Index']=pd.to_datetime(data['Index'])
data = data.drop(['OPEN', 'HIGH', 'LOW'], axis=1) 

data_2018 = data[499:-251] #1/02/2009 - 12/29/2017    
data = data[499:] #1/02/2009 - 12/31/2018 
data.tail()

,Index,R,US10YR,EPS,PER,CLOSE,BDIY,VIX,PCR,MVOLE,...,MYSTERYVAR3,TOM,HULBERTSENTIMENT,BUBBLE,CASH,DEBT,DIVIDEND,RETURN,STOCKS,WAR
16858,2018-12-24,0.022629,2.7383,132.51,17.696918,2351.10,1271.0,36.07,0.79,11037.81,...,12.133489,0,-11.74,61.833407,145.063192,41.207903,76.017489,32.030824,51.823052,31.871601
16859,2018-12-26,0.007556,2.8079,132.50,18.622795,2467.70,1271.0,30.41,0.62,19311.44,...,12.338862,1,-14.94,55.718015,120.402449,29.434217,55.868275,28.959375,44.785354,32.472952
16860,2018-12-27,-0.001382,2.7666,132.51,18.778925,2488.83,1271.0,29.96,0.73,17158.61,...,12.115016,1,-15.59,67.948799,121.853081,36.498429,57.700022,39.051278,47.344517,37.283760
16861,2018-12-28,0.008630,2.7182,132.51,18.758886,2485.74,1271.0,28.34,0.44,15667.72,...,12.042282,1,-15.59,61.833407,124.754345,41.796588,73.269869,39.490056,51.823052,36.682409
16862,2018-12-31,0.000947,2.6842,132.52,18.919920,2506.85,1271.0,25.42,0.60,15284.06,...,12.067827,1,-11.74,61.153919,137.810032,42.973956,79.680982,36.857386,47.344517,36.682409


In [3]:
df_prophet=data_2018[['Index','R']]
df_prophet=df_prophet.sort_values('Index')
df_prophet=df_prophet.rename(columns={'Index':'ds','R':'y'})

df_prophet.tail(3)

,ds,y
16609,2017-12-27,0.002001
16610,2017-12-28,-0.003831
16611,2017-12-29,0.007078


In [4]:
m=Prophet()
m.fit(df_prophet)


future=m.make_future_dataframe(periods=365)

future.tail()
forecast=m.predict(future)
forecast.head(5)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2009-01-02,0.000392,-0.010486,0.011941,0.000392,0.000392,-0.000176,-0.000176,-0.000176,0.000234,0.000234,0.000234,-0.000410,-0.000410,-0.000410,0.0,0.0,0.0,0.000216
1,2009-01-05,0.000391,-0.012516,0.011376,0.000391,0.000391,-0.000656,-0.000656,-0.000656,0.000353,0.000353,0.000353,-0.001009,-0.001009,-0.001009,0.0,0.0,0.0,-0.000265
2,2009-01-06,0.000391,-0.011199,0.011237,0.000391,0.000391,-0.000732,-0.000732,-0.000732,0.000473,0.000473,0.000473,-0.001204,-0.001204,-0.001204,0.0,0.0,0.0,-0.000340
3,2009-01-07,0.000391,-0.011770,0.011023,0.000391,0.000391,-0.000893,-0.000893,-0.000893,0.000498,0.000498,0.000498,-0.001391,-0.001391,-0.001391,0.0,0.0,0.0,-0.000502
4,2009-01-08,0.000391,-0.012245,0.010491,0.000391,0.000391,-0.001542,-0.001542,-0.001542,0.000021,0.000021,0.000021,-0.001563,-0.001563,-0.001563,0.0,0.0,0.0,-0.001151


In [14]:
results = forecast.drop(['trend', 'yhat_lower', 'yhat_upper','trend_lower', 'trend_upper', 'additive_terms', 'additive_terms_lower',
                        'additive_terms_upper', 'weekly','weekly_lower','weekly_upper','yearly','yearly_lower','yearly_upper',
                        'multiplicative_terms','multiplicative_terms_lower','multiplicative_terms_upper'], axis=1) 
results = results[:-1]


results.rename(columns = {'yhat':'PREDICTED'}, inplace = True) 
results.set_index('ds').head(5)

,PREDICTED
ds,
2009-01-02,0.000625
2009-01-05,0.000579
2009-01-06,0.000466
2009-01-07,0.000303
2009-01-08,-0.000388


# Check that Predicted R-Average for 2018 is similar to actual R-average (0.0008706)

In [15]:
R_values = data[:-1]
R_values.drop(['US10YR','ASP', 'TERM', 'VRP', 
         'LOAN','DXY', 'HTIME', 'LTIME', 'IC', 
         'INF', 'UR', 'URR', 'HS', 'FOMC', 'PPIR', 
         'NAPMPMI', 'NAPMNEWO', 'NAPMPRIC', 'PTB', 'CATY','PCR', 'NOS', 'MVOLE', 'NYSEADV', 'RV',
         'RELINF', 'SIM', 'TOM', 'BDIY', 'SENTIMENT1',
         'SENTIMENT2', 'SENTIMENT3', 'SENTIMENT4', 'HULBERTSENTIMENT',
         'MYSTERYVAR1', 'MYSTERYVAR2','DEBT', 'RETURN', 'STOCKS', 'WAR','EPS','PER','CLOSE','VIX','ADVDECL', 'BAA',
              'BER', 'DEL', 'BBY', 'TYVIX', 'CRP', 'INDPRO', 'CAPE', 'FEDFUNDS', 'DVY', 'AAA', 'SI',
              'VVIX', 'US3M', 'MYSTERYVAR3', 'CASH', 'DIVIDEND', 'BUBBLE'], axis=1)
R_values=R_values[['Index','R']].sort_values('Index')
R_values=R_values.rename(columns={'Index':'ds','R':'y'})
R_values.set_index('ds')
Copy_Target = R_values


result = results.merge(R_values, on='ds', how='inner') 
result.rename(columns={'ds':'date','PREDICTED':'prediction', 'y':'target'}, inplace=True)
result.head(10)


,date,prediction,target
0,2009-01-02,0.000625,-0.001187
1,2009-01-05,0.000579,0.006650
2,2009-01-06,0.000466,-0.022717
3,2009-01-07,0.000303,0.004069
4,2009-01-08,-0.000388,-0.021655
5,2009-01-09,-0.000263,-0.022731
6,2009-01-12,-0.000332,0.001834
7,2009-01-13,-0.000406,-0.022745
8,2009-01-14,-0.000508,0.000352
9,2009-01-15,-0.001116,0.007785


In [41]:
prediction_avg = mean(result['prediction'])
target_avg = mean(result['target'])

percent_err = (abs(target_avg - prediction_avg)/(target_avg)) * 100

print("Average Predicted R value for 2018:", prediction_avg) 
print("Average Actual R Value for 2018:", target_avg)
print("Average Error:", percent_err, "%\n")
R2 = r2_score(result['target'], result['prediction'])
score = 0.00087

print("Benchmark R^2 ----->", score)
print("Our R^2 Score ----->", round(R2,5))


Average Predicted R value for 2018: 0.0005209756146929783
Average Actual R Value for 2018: 0.000534048394494265
Average Error: 2.447864263999228 %

Benchmark R^2 -----> 0.00087
Our R^2 Score -----> 0.00746


# Prophet Model for test 2: Concat 2019 data: [date, target(R Values)] to our 2009-2018 Train Set

In [17]:
prophet_2019 =  data[['Index','R']]
prophet_2019=prophet_2019.sort_values('Index')
prophet_2019=prophet_2019.rename(columns={'Index':'ds','R':'y'})

prophet_2019.tail(3)

,ds,y
16860,2018-12-27,-0.001382
16861,2018-12-28,0.008630
16862,2018-12-31,0.000947


Hold = 1,734
Prophet = 1,858
Prophet Thresh = 1,399
Seasonality = 1,953
Optimized Prophet Thresh = 1,663

In [18]:
m=Prophet()
m.fit(prophet_2019)


future=m.make_future_dataframe(periods=365)

future.tail()
forecast=m.predict(future)
forecast.tail(5)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
2876,2019-12-27,-0.00001,-0.011214,0.011788,-0.000010,-0.00001,0.000468,0.000468,0.000468,0.000189,0.000189,0.000189,0.000279,0.000279,0.000279,0.0,0.0,0.0,0.000458
2877,2019-12-28,-0.00001,-0.012113,0.011378,-0.000010,-0.00001,-0.000497,-0.000497,-0.000497,-0.000792,-0.000792,-0.000792,0.000295,0.000295,0.000295,0.0,0.0,0.0,-0.000507
2878,2019-12-29,-0.00001,-0.011533,0.010864,-0.000010,-0.00001,-0.000498,-0.000498,-0.000498,-0.000792,-0.000792,-0.000792,0.000294,0.000294,0.000294,0.0,0.0,0.0,-0.000508
2879,2019-12-30,-0.00001,-0.009521,0.012465,-0.000011,-0.00001,0.000758,0.000758,0.000758,0.000484,0.000484,0.000484,0.000274,0.000274,0.000274,0.0,0.0,0.0,0.000748
2880,2019-12-31,-0.00001,-0.010338,0.012154,-0.000011,-0.00001,0.000741,0.000741,0.000741,0.000505,0.000505,0.000505,0.000236,0.000236,0.000236,0.0,0.0,0.0,0.000731


In [19]:
results = forecast.drop(['trend', 'yhat_lower', 'yhat_upper','trend_lower', 'trend_upper', 'additive_terms', 'additive_terms_lower',
                        'additive_terms_upper', 'weekly','weekly_lower','weekly_upper','yearly','yearly_lower','yearly_upper',
                        'multiplicative_terms','multiplicative_terms_lower','multiplicative_terms_upper'], axis=1) 
results = results[:-3]



results.rename(columns = {'yhat':'PREDICTED'}, inplace = True) 
results.set_index('ds')

target_same = Copy_Target[1762:]
target_same['ds'] = target_same['ds'].astype('datetime64') + pd.offsets.DateOffset(years=1)





/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [23]:
results = results.merge(target_same, on='ds', how='inner') 
results.rename(columns={'ds':'date','PREDICTED':'prediction', 'y':'target'}, inplace=True)

KeyError: 'ds'

In [28]:
results.tail(20)

,date,prediction,target
618,2019-11-29,0.000399,0.005986
619,2019-11-30,-0.000648,0.013065
620,2019-12-03,0.000415,-0.022066
621,2019-12-04,0.000308,-0.001611
622,2019-12-06,-0.000134,-0.022072
623,2019-12-07,-0.001181,0.001804
624,2019-12-10,-0.000024,0.000134
625,2019-12-11,-0.000070,0.004930
626,2019-12-12,-0.000629,-0.000432
627,2019-12-13,-0.000359,-0.018730


In [43]:
R2 = r2_score(results['target'], results['prediction'])
prediction_2019 = mean(results['prediction'])
score = -0.0162

print("Benchmark R^2 ----->", score)
print("Our R^2 Score ----->", round(R2,5))
print("R Average Prediction for 2019:", prediction_2019)

Benchmark R^2 -----> -0.0162
Our R^2 Score -----> 0.00181
R Average Prediction for 2019: 0.00021720804839558781


In [44]:
buys = 1
sells = 0
output = open("output.txt","w")

output.write("%s\n" % ((prediction_2019)))
output.write("%d\n" % (buys))
output.write('%d\n' % sells)
output.close()